In [3]:
csv_path = "C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/heavy-stocks.csv"
df_symbols = pd.read_csv(csv_path)
symbols = df_symbols['Symbol'].dropna().unique().tolist()


In [4]:
symbols

['NVDA',
 'MSFT',
 'AAPL',
 'AMZN',
 'GOOG',
 'GOOGL',
 'META',
 'TSLA',
 'AVGO',
 'BRK.B',
 'WMT',
 'JPM',
 'V',
 'LLY',
 'MA',
 'NFLX',
 'ORCL',
 'COST',
 'XOM',
 'PG',
 'JNJ',
 'HD',
 'BAC',
 'ABBV',
 'KO',
 'PLTR',
 'PM',
 'TMUS',
 'UNH',
 'GE',
 'CRM',
 'CSCO',
 'IBM',
 'WFC',
 'CVX',
 'ABT',
 'MCD',
 'LIN',
 'NOW',
 'INTU',
 'AXP',
 'MS',
 'DIS',
 'ISRG',
 'ACN',
 'T',
 'MRK',
 'GS',
 'AMD',
 'VZ',
 'PEP',
 'RTX',
 'BKNG',
 'UBER',
 'ADBE',
 'TXN',
 'CAT',
 'QCOM',
 'PGR',
 'SCHW',
 'SPGI',
 'BA',
 'BSX',
 'TMO',
 'AMGN',
 'BLK',
 'SYK',
 'HON',
 'TJX',
 'NEE',
 'C',
 'GILD',
 'DE',
 'DHR',
 'PFE',
 'UNP',
 'ADP',
 'GEV',
 'AMAT',
 'ETN',
 'CMCSA',
 'LOW',
 'PANW',
 'COF',
 'CB',
 'CRWD',
 'VRTX',
 'MMC',
 'LMT',
 'ANET',
 'MU',
 'KKR',
 'LRCX',
 'APH',
 'COP',
 'ADI',
 'BX',
 'MDT',
 'KLAC',
 'CME',
 'ICE',
 'PLD',
 'MO',
 'WELL',
 'AMT',
 'SO',
 'TT',
 'BMY',
 'SBUX',
 'CEG',
 'WM',
 'HCA',
 'NKE',
 'CTAS',
 'DUK',
 'SHW',
 'MCK',
 'FI',
 'INTC',
 'MDLZ',
 'AJG',
 'DASH',
 'EQI

In [1]:
# 📦 Import required libraries
import time
import logging
from pathlib import Path
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from tenacity import retry, stop_after_attempt, wait_exponential

# 🔧 Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
API_KEY = "9HKRTHJGP77FGDQO"  # Replace with your own key if needed
OUTPUT_DIR = Path("av_monthly")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

csv_path = "C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/heavy-stocks.csv"
df_symbols = pd.read_csv(csv_path)
symbols = df_symbols['Symbol'].dropna().unique().tolist()

print(f"Found {len(symbols)} symbols:")
print(symbols[:5])  # Show a preview

ts = TimeSeries(key=API_KEY, output_format='pandas')

ticker="MSFT"
logger.info(f"Fetching data for {ticker}")
data, _ = ts.get_monthly_adjusted(symbol=ticker)

if data.empty:
    raise ValueError(f"No data returned for {ticker}")

data.reset_index(inplace=True)
data.rename(columns={
    'date': 'Date',
    '1. open': 'Open',
    '2. high': 'High',
    '3. low': 'Low',
    '4. close': 'Close',
    '5. adjusted close': 'Adj Close',
    '6. volume': 'Volume'
}, inplace=True)

data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
data['Date'] = pd.to_datetime(data['Date']).dt.strftime('%Y-%m-%d')
data = data.sort_values('Date')
data = data.round(4)


2025-05-29 13:24:50,932 - INFO - Fetching data for MSFT


Found 503 symbols:
['NVDA', 'MSFT', 'AAPL', 'AMZN', 'GOOG']


ValueError: We have detected your API key as 9HKRTHJGP77FGDQO and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.

In [4]:
!pip install polygon-api-client

In [6]:
!where python


C:\Python\python.exe
C:\Users\Steel\AppData\Local\Microsoft\WindowsApps\python.exe
C:\msys64\ucrt64\bin\python.exe
C:\Panda3D-1.10.15-x64\python\python.exe


In [24]:
# Polygon.io setup
from polygon import RESTClient
import pandas as pd
import time
import os
from pathlib import Path
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Replace with your Polygon API Key
POLYGON_API_KEY = "V14jfLkPqCx651gIIRS0ipNy5m8BH0xS"
client = RESTClient(POLYGON_API_KEY)

# Tickers to fetch
tickers = ['AAPL', 'MSFT', 'GOOG']

# Output directory
output_dir = Path("polygon_monthly")
output_dir.mkdir(parents=True, exist_ok=True)

# Fetch and save monthly data
for ticker in tickers:
    logging.info(f"Fetching monthly data for {ticker}")
    try:
        bars = client.get_aggs(
            ticker=ticker,
            multiplier=1,
            timespan="month",
            from_="2000-01-01",
            to="2025-05-29",
            limit=5000
        )
        df = pd.DataFrame([bar.__dict__ for bar in bars])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[['timestamp', 'close']]
        df = df.round(4)
        df.sort_values('timestamp', inplace=True)
        df.rename(columns={'timestamp': 'date'}, inplace=True)
        df.set_index('date',inplace=True)
        df.to_csv(output_dir / f"{ticker}_monthly.csv", index=True)
        logging.info(f"Saved {ticker} to {output_dir}/{ticker}_monthly.csv")
        time.sleep(1)
    except Exception as e:
        logging.error(f"Failed to fetch {ticker}: {e}")

2025-05-29 13:56:41,483 - INFO - Fetching monthly data for AAPL
2025-05-29 13:56:41,885 - INFO - Saved AAPL to polygon_monthly/AAPL_monthly.csv
2025-05-29 13:56:42,886 - INFO - Fetching monthly data for MSFT
2025-05-29 13:56:42,946 - INFO - Saved MSFT to polygon_monthly/MSFT_monthly.csv
2025-05-29 13:56:43,947 - INFO - Fetching monthly data for GOOG
2025-05-29 13:56:44,688 - ERROR - Failed to fetch GOOG: HTTPSConnectionPool(host='api.polygon.io', port=443): Max retries exceeded with url: /v2/aggs/ticker/GOOG/range/1/month/2000-01-01/2025-05-29?limit=5000 (Caused by ResponseError('too many 429 error responses'))


In [11]:
bars = client.get_aggs(
    ticker=ticker,
    multiplier=1,
    timespan="month",
    from_="2000-01-01",
    to="2025-05-29",
    limit=5000
)
df = pd.DataFrame([bar.__dict__ for bar in bars])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df[['timestamp', 'close']]
df = df.round(4)
df.sort_values('timestamp', inplace=True)
df.rename(columns={'timestamp': 'date'}, inplace=True)
df.set_index('date',inplace=True)

In [2]:
import yfinance as yf
import pandas as pd

ticker = "AAPL"
try:
    print(f"Testing {ticker}...")
    stock = yf.Ticker(ticker)
    df = stock.history(start="2000-01-01", end="2025-05-29", interval="1mo")
    if not df.empty:
        df = df[['Close']].round(4)
        df.to_csv(f"{ticker}_monthly_yf.csv")
        print(f"Success! Saved data to {ticker}_monthly_yf.csv")
        print(df.head())
    else:
        print(f"No data for {ticker}")
except Exception as e:
    print(f"Error: {e}")

Testing AAPL...
Success! Saved data to AAPL_monthly_yf.csv
                            Close
Date                             
2000-01-01 00:00:00-05:00  0.7795
2000-02-01 00:00:00-05:00  0.8612
2000-03-01 00:00:00-05:00  1.0204
2000-04-01 00:00:00-05:00  0.9321
2000-05-01 00:00:00-04:00  0.6311


In [21]:
df

,close
date,
2023-05-01 04:00:00,123.37
2023-06-01 04:00:00,120.97
2023-07-01 04:00:00,133.11
2023-08-01 04:00:00,137.35
2023-09-01 04:00:00,131.85
2023-10-01 04:00:00,125.30
2023-11-01 04:00:00,133.92
2023-12-01 05:00:00,140.93
2024-01-01 05:00:00,141.80


In [ ]:
V14jfLkPqCx651gIIRS0ipNy5m8BH0xS

d0s9i2hr01qkkpltrpe0d0s9i2hr01qkkpltrpeg


In [7]:
!pip install finnhub-python


  Obtaining dependency information for finnhub-python from https://files.pythonhosted.org/packages/17/09/9240b2a222717e7bda81f954047b662f2744aaeb6b29d62e89bb5c49dd16/finnhub_python-2.4.23-py3-none-any.whl.metadata


In [17]:
# Finnhub.io setup
import finnhub
import pandas as pd
import time
import os
from pathlib import Path
import logging
from datetime import datetime
import calendar

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Replace with your Finnhub API Key
FINNHUB_API_KEY = "d0s9ll9r01qkkpltsgigd0s9ll9r01qkkpltsgj0"
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)


In [18]:
finnhub_client

In [19]:
res = finnhub_client.stock_candles('AAPL', 'D', 1590988249, 1591852249)
print(res)

FinnhubAPIException: FinnhubAPIException(status_code: 403): You don't have access to this resource.

In [25]:
# Finnhub.io setup
import finnhub
import pandas as pd
import time
import os
from pathlib import Path
import logging
from datetime import datetime
import calendar

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Replace with your Finnhub API Key
FINNHUB_API_KEY = "d0s9i2hr01qkkpltrpe0d0s9i2hr01qkkpltrpeg"
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# Tickers to fetch
tickers = ['AAPL', 'MSFT', 'GOOG']

# Output directory
output_dir = Path("finnhub_monthly")
output_dir.mkdir(parents=True, exist_ok=True)

def get_month_start_end_dates(year: int, month: int):
    first_day = datetime(year, month, 1)
    last_day = datetime(year, month, calendar.monthrange(year, month)[1])
    return int(first_day.timestamp()), int(last_day.timestamp())

# Fetch and save monthly data
for ticker in tickers:
    logging.info(f"Fetching monthly data for {ticker}")
    try:
        all_months = []
        for year in range(2015, 2025):
            for month in range(1, 13):
                start, end = get_month_start_end_dates(year, month)
                candles = finnhub_client.stock_candles(ticker, 'D', start, end)
                if candles.get('s') != 'ok':
                    continue
                df = pd.DataFrame(candles)
                df['t'] = pd.to_datetime(df['t'], unit='s')
                monthly_df = df.groupby(df['t'].dt.to_period('M')).agg({
                    'o': 'first',
                    'h': 'max',
                    'l': 'min',
                    'c': 'last',
                    'v': 'sum'
                }).reset_index()
                all_months.append(monthly_df)

        if all_months:
            final_df = pd.concat(all_months)
            final_df.rename(columns={
                't': 'Date',
                'o': 'Open',
                'h': 'High',
                'l': 'Low',
                'c': 'Close',
                'v': 'Volume'
            }, inplace=True)
            final_df['Date'] = final_df['Date'].astype(str)
            final_df = final_df.round(4)
            final_df.to_csv(output_dir / f"{ticker}_monthly.csv", index=False)
            logging.info(f"Saved {ticker} to {output_dir}/{ticker}_monthly.csv")
        time.sleep(1)
    except Exception as e:
        logging.error(f"Failed to fetch {ticker}: {e}")


2025-05-29 13:59:07,359 - INFO - Fetching monthly data for AAPL
2025-05-29 13:59:07,825 - ERROR - Failed to fetch AAPL: FinnhubAPIException(status_code: 401): Invalid API key
2025-05-29 13:59:07,826 - INFO - Fetching monthly data for MSFT
2025-05-29 13:59:07,864 - ERROR - Failed to fetch MSFT: FinnhubAPIException(status_code: 401): Invalid API key
2025-05-29 13:59:07,864 - INFO - Fetching monthly data for GOOG
2025-05-29 13:59:07,904 - ERROR - Failed to fetch GOOG: FinnhubAPIException(status_code: 401): Invalid API key


In [9]:
from polygon import RESTClient

In [8]:
!pip install polygon-api-client

  Obtaining dependency information for polygon-api-client from https://files.pythonhosted.org/packages/fb/16/ee8a361c40432408684113e90fba6fe6a862ffd8a7c518be1167479e8cad/polygon_api_client-1.14.5-py3-none-any.whl.metadata
  Using cached polygon_api_client-1.14.5-py3-none-any.whl.metadata (952 bytes)
Using cached polygon_api_client-1.14.5-py3-none-any.whl (44 kB)


In [32]:
!pip install yfinance --upgrade

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/73/b5/d50eec88bc731bb8570ae42a9b764a36144e217361c33fa068391ff59ba3/yfinance-0.2.61-py2.py3-none-any.whl.metadata
  Obtaining dependency information for curl_cffi>=0.7 from https://files.pythonhosted.org/packages/d9/f2/a46509f4f3054bbbaae822c74145bc7377ef3ca3b12c3f2bb82314d81b2a/curl_cffi-0.11.1-cp39-abi3-win_amd64.whl.metadata
  Obtaining dependency information for protobuf>=3.19.0 from https://files.pythonhosted.org/packages/44/3a/b15c4347dd4bf3a1b0ee882f384623e2063bb5cf9fa9d57990a4f7df2fb6/protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata
  Obtaining dependency information for websockets>=13.0 from https://files.pythonhosted.org/packages/98/93/e36c73f78400a65f5e236cd376713c34182e6663f6889cd45a4a04d8f203/websockets-15.0.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for certifi>=2024.2.2 from https://files.pythonhosted.org/packages/4a/7e/3db2bd1b1f9e95f7cddca6d6e75e2f2

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Python\\Lib\\site-packages\\~ebsockets\\speedups.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [34]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time

# Function to get S&P 500 tickers (optional, for scaling to all S&P 500 stocks)
def get_sp500_tickers():
    try:
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        tables = pd.read_html(url)
        sp500_table = tables[0]
        tickers = sp500_table['Symbol'].tolist()
        # Replace problematic tickers (e.g., BRK.B to BRK-B)
        return [ticker.replace('.', '-') for ticker in tickers]
    except Exception as e:
        print(f"Error fetching S&P 500 tickers: {e}")
        return []

# Function to fetch historical data
def fetch_historical_data(tickers, start_date, end_date, interval='1mo'):
    for ticker in tickers:
        try:
            print(f"Fetching data for {ticker}...")
            # Use yf.download for consistency with your original script
            df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
            if not df.empty:
                # Keep only adjusted close prices and round to 4 decimal places
                df = df[['Adj Close']].rename(columns={'Adj Close': ticker}).round(4)
                # Save to CSV
                df.to_csv(f"{ticker}_monthly_yf.csv")
                print(f"Saved data for {ticker} to {ticker}_monthly_yf.csv")
            else:
                print(f"No data found for {ticker}")
            # Avoid rate-limiting
            time.sleep(0.5)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

# Main execution
if __name__ == "__main__":
    # Define date range (5 years ago to today, avoiding future dates)
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=5*365)  # 5 years ago

    # Use your test tickers or fetch all S&P 500 tickers
    tickers = ['AAPL', 'MSFT', 'GOOG']  # Replace with get_sp500_tickers() for all S&P 500
    # tickers = get_sp500_tickers()

    if not tickers:
        print("No tickers to process. Exiting.")
        exit()

    print(f"Processing {len(tickers)} tickers: {tickers[:5]}{'...' if len(tickers) > 5 else ''}")

    # Fetch historical data
    fetch_historical_data(tickers, start_date, end_date, interval='1mo')

Processing 3 tickers: ['AAPL', 'MSFT', 'GOOG']
Fetching data for AAPL...
[*********************100%%**********************]  1 of 1 completed

2025-05-29 14:04:09,768 - ERROR - 
1 Failed download:
2025-05-29 14:04:09,769 - ERROR - ['AAPL']: Exception('%ticker%: No price data found, symbol may be delisted (1mo 2020-05-30 -> 2025-05-29)')



No data found for AAPL
Fetching data for MSFT...
[*********************100%%**********************]  1 of 1 completed

2025-05-29 14:04:11,836 - ERROR - 
1 Failed download:
2025-05-29 14:04:11,836 - ERROR - ['MSFT']: Exception('%ticker%: No price data found, symbol may be delisted (1mo 2020-05-30 -> 2025-05-29)')



No data found for MSFT
Fetching data for GOOG...
[*********************100%%**********************]  1 of 1 completed

2025-05-29 14:04:13,925 - ERROR - 
1 Failed download:
2025-05-29 14:04:13,926 - ERROR - ['GOOG']: Exception('%ticker%: No price data found, symbol may be delisted (1mo 2020-05-30 -> 2025-05-29)')



No data found for GOOG


In [36]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Test a single ticker
ticker = "AAPL"
start_date = "2020-05-01"
end_date = "2025-05-29"

try:
    print(f"Fetching data for {ticker}...")
    df = yf.download(ticker, start=start_date, end=end_date, interval="1d", auto_adjust=True)
    if not df.empty:
        df = df[['Close']].round(4)
        df.to_csv(f"{ticker}_monthly_yf.csv")
        print(f"Data saved to {ticker}_monthly_yf.csv")
        print(df.head())
    else:
        print(f"No data found for {ticker}")
except Exception as e:
    print(f"Error: {e}")

Fetching data for AAPL...
[*********************100%%**********************]  1 of 1 completed

2025-05-29 14:05:15,412 - ERROR - 
1 Failed download:
2025-05-29 14:05:15,413 - ERROR - ['AAPL']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2020-05-01 -> 2025-05-29)')



No data found for AAPL


In [38]:
import requests
data = requests.get("http://chart.finance.yahoo.com/table.csv?s=MSFT&a=2&b=15&c=2015&d=4&e=15&f=2017&g=d&ignore=.csv")

ConnectionError: HTTPConnectionPool(host='chart.finance.yahoo.com', port=80): Max retries exceeded with url: /table.csv?s=MSFT&a=2&b=15&c=2015&d=4&e=15&f=2017&g=d&ignore=.csv (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000204E47B6D90>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [39]:
!pip install --upgrade yfinance

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/73/b5/d50eec88bc731bb8570ae42a9b764a36144e217361c33fa068391ff59ba3/yfinance-0.2.61-py2.py3-none-any.whl.metadata
  Using cached yfinance-0.2.61-py2.py3-none-any.whl.metadata (5.8 kB)
  Obtaining dependency information for curl_cffi>=0.7 from https://files.pythonhosted.org/packages/d9/f2/a46509f4f3054bbbaae822c74145bc7377ef3ca3b12c3f2bb82314d81b2a/curl_cffi-0.11.1-cp39-abi3-win_amd64.whl.metadata
  Using cached curl_cffi-0.11.1-cp39-abi3-win_amd64.whl.metadata (15 kB)
  Obtaining dependency information for protobuf>=3.19.0 from https://files.pythonhosted.org/packages/44/3a/b15c4347dd4bf3a1b0ee882f384623e2063bb5cf9fa9d57990a4f7df2fb6/protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata
  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Obtaining dependency information for certifi>=2024.2.2 from https://files.pythonhosted.org/packages/4a/7e/3db2bd1b1f9e95f7cddca6d6e75e2f2b

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
polygon-api-client 1.14.5 requires websockets<15.0,>=10.3, but you have websockets 15.0.1 which is incompatible.
pyppeteer 2.0.0 requires websockets<11.0,>=10.0, but you have websockets 15.0.1 which is incompatible.


In [50]:
msft = yf.Ticker("MSFT")

In [51]:
msft.financials

""


In [44]:
from pandas_datareader import data as pdr

ModuleNotFoundError: No module named 'pandas_datareader'

In [45]:
!pip install pandas_datareader

  Obtaining dependency information for pandas_datareader from https://files.pythonhosted.org/packages/3f/16/56c9d648b503619ebe96f726b5f642b68e299b34162ed2d6faa9d7966b7d/pandas_datareader-0.10.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/109.5 kB ? eta -:--:--
   --- ------------------------------------ 10.2/109.5 kB ? eta -:--:--
   -------------- ------------------------ 41.0/109.5 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 109.5/109.5 kB 1.3 MB/s eta 0:00:00


In [46]:
from pandas_datareader import data as pdr

In [48]:
data = pdr.get_data_yahoo("MSFT", start="2017-01-01", end="2017-04-30")

AttributeError: 'NoneType' object has no attribute 'group'

In [49]:
pip show yfinance

Name: yfinanceNote: you may need to restart the kernel to use updated packages.

Version: 0.2.61
Summary: Download market data from Yahoo! Finance API
Home-page: https://github.com/ranaroussi/yfinance
Author: Ran Aroussi
Author-email: ran@aroussi.com
License: Apache
Location: C:\Python\Lib\site-packages
Requires: beautifulsoup4, curl_cffi, frozendict, multitasking, numpy, pandas, peewee, platformdirs, protobuf, pytz, requests, websockets
Required-by: 
